Discussion

1) the motivation behind the presented solutions
2) how you represent and preprocess the data
3) your idea and solution implementation for each of the two tasks listed above.
4) the evaluation procedure (e.g., cross-validation or training/validation split)
5) the training/validation results. 

In [10]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


import pandas as pd
import numpy as np



In [11]:
k_folds = 10

In [5]:
yelp_df = pd.read_csv('../data/input/train.csv')

Implement Naive Bayes using Unpreprocessed Data

In [15]:
# create a bag of words representation of the review text
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(yelp_df['review'])

# label column
y = yelp_df['category']

# train the Naive Bayes model with 10-fold cross-validation
clf = MultinomialNB()
scores = cross_val_score(clf, X, y, cv=k_folds)

# calculate the average accuracy and standard deviation of the model
print('Average accuracy:', round(scores.mean() * 100, 4), " %")
print('Standard deviation:', round(scores.std() * 100, 4), " %")

Average accuracy: 87.1032  %
Standard deviation: 1.3601  %
